# Upload Raspberry Pi with AWS
- Part 1: Load data from Arduino and store data incorrect format
- Part 2: Connect Raspberry with S3 by boto3

#### Part 1

In [19]:
import serial                   # Python library to read data from Raspberry Pi serial 
import time
import bluetooth
from datetime import datetime   # Python library to check datetime
import os                       # Python library to use operating system dependent functionality

The steps to connect and read data of Arduino through USB serial can be found in [this website](https://oscarliang.com/connect-raspberry-pi-and-arduino-usb-cable/).
The serial number in Arduino code should match the serial number of Raspeberry-pi code. The USB interface can be see by this linux command `ls /dev/tty*` 

In [2]:
!ls /dev/tty*

/dev/tty    /dev/tty19	/dev/tty3   /dev/tty40	/dev/tty51  /dev/tty62
/dev/tty0   /dev/tty2	/dev/tty30  /dev/tty41	/dev/tty52  /dev/tty63
/dev/tty1   /dev/tty20	/dev/tty31  /dev/tty42	/dev/tty53  /dev/tty7
/dev/tty10  /dev/tty21	/dev/tty32  /dev/tty43	/dev/tty54  /dev/tty8
/dev/tty11  /dev/tty22	/dev/tty33  /dev/tty44	/dev/tty55  /dev/tty9
/dev/tty12  /dev/tty23	/dev/tty34  /dev/tty45	/dev/tty56  /dev/ttyAMA0
/dev/tty13  /dev/tty24	/dev/tty35  /dev/tty46	/dev/tty57  /dev/ttyprintk
/dev/tty14  /dev/tty25	/dev/tty36  /dev/tty47	/dev/tty58
/dev/tty15  /dev/tty26	/dev/tty37  /dev/tty48	/dev/tty59
/dev/tty16  /dev/tty27	/dev/tty38  /dev/tty49	/dev/tty6
/dev/tty17  /dev/tty28	/dev/tty39  /dev/tty5	/dev/tty60
/dev/tty18  /dev/tty29	/dev/tty4   /dev/tty50	/dev/tty61


In [3]:
# Read from the serial
try:
    read_serial = serial.Serial("/dev/ttyUSB0", 115200) # 115200 is the serial number in Arduino code
                                                      # dev/ttyUSB0 is the port number optain from ls/dev/tty*
except:
    read_serial = serial.Serial("/dev/ttyUSB1", 115200)

SerialException: could not open port /dev/ttyUSB1: [Errno 2] No such file or directory: '/dev/ttyUSB1'

In [2]:
# Create file name which match the date
current_time = datetime.now()
file_name = current_time.strftime("%Y-%m-%d") + '.csv'
file_name

'2018-01-24.csv'

In [3]:
# The user's name is also the folder's name on s3
user = 'Vu'

In [4]:
# Create columns for the data file
def print_first_line(written_file):
    written_file.write('Date,')
    written_file.write('Patient ID,')
    written_file.write('Red Signal')
    written_file.write('\n')
    return written_file

In [5]:
def check_file_exist(file_name):
    if file_name not in os.listdir(os.curdir):
        # File does not exist, create new file
        written_file = open(file_name, "w")
        written_file = print_first_line(written_file)
    else:
        # File already exist, append new rows
        written_file = open(file_name, "a")
    return written_file

In [13]:
# Write to file locally
import time

print("Read data from Serial")
file_signal = check_file_exist(file_name)

start = time.time()
stop = time.time()
try: 
    while (stop - start) < 3:
        data = read_serial.readline()  # Data from serial separate by newline character
        data = data.strip()            # Remove newline character of Arduino IDE 
        file_signal.write(datetime.now().strftime("%H:%M:%S.%f") + ",")    # Record time corresponding to the signal 
        file_signal.write(user+ ",")    # Write to column Patient Id
        file_signal.write(data)
        file_signal.write("\n")
        stop = time.time()
except KeyboardInterrupt:
    file_signal.close()
    print("End")

Read data from Serial


NameError: name 'read_serial' is not defined

In [30]:
!head 2018-01-12.csv

Date,Patient ID,Red Signal
10:12:30.736319,Vu,Initializing MAX30100..SUCCESS
10:12:30.736677,Vu,1680
10:12:30.736904,Vu,1890
10:12:30.737102,Vu,1889
10:12:30.737302,Vu,1893
10:12:30.737499,Vu,1896
10:12:30.737697,Vu,1891
10:12:30.737895,Vu,1895
10:12:30.738090,Vu,1893


#### Part 2

In [6]:
import boto3
import botocore

BUCKET_NAME = 'canaria-client-data'

In [7]:
# Print all file inside S3 bucket
session = boto3.Session(profile_name='default') # take secret key in awscli credentials file.
s3_client = session.client('s3')
for key in s3_client.list_objects(Bucket=BUCKET_NAME)['Contents']:
    print(key['Key'])

2018-01-02.csv
DockerToolbox.exe
Khoi/2018-01-17.csv
Khoi/2018-01-18.csv
Vu/2018-01-22.csv
client/test-client-1/1516604740790.json
client/test-client-1/1516604748629.json


In [8]:
# Check file exist in S3
for key in s3_client.list_objects(Bucket=BUCKET_NAME)['Contents']:
    if key['Key'] == 'empty/2018-01-02.csv':
        print('Exists')
    else:
        print("Doesn't exist")

Doesn't exist
Doesn't exist
Doesn't exist
Doesn't exist
Doesn't exist
Doesn't exist
Doesn't exist


In [9]:
# Upload file to s3
s3_client.upload_file(file_name, BUCKET_NAME, 'empty/'+file_name)

OSError: [Errno 2] No such file or directory: '2018-01-23.csv'

In [10]:
# Download file to Raspberry-pi 
s3_client.download_file(BUCKET_NAME, '2018-01-02.csv', '2018-01-02.csv')

## Finalize
The program will check if the file exists on s3. It the file exists, the program will download and write to the file. If s3 does not have the file, the program will create locally then upload on s3.

In [6]:
# Make connection with s3 aws
import boto3
import botocore

BUCKET_NAME = 'canaria-client-data'
session = boto3.Session(profile_name='default') # take secret key in awscli credentials file.
s3_client = session.client('s3')

In [7]:
def exist_file_from_s3(file_name, user, bucket_name, s3_client):
    exist_file = 0
    for key in s3_client.list_objects(Bucket=bucket_name)['Contents']:
        if key['Key'] == (user+'/'+file_name):
            print('downloading file...')
            s3_client.download_file(bucket_name, user+'/' + file_name, file_name)
            print('opening the file...')
            exist_file = open(file_name, "a")
            break
    if exist_file == 0:
        print('Making file...')
        exist_file = check_file_exist(file_name)
    print('Finished!')
    return exist_file

In [8]:
def update_file_signal(file_write, file_name, user, bucket_name, s3_client):
    file_write.close()                                       # Close the file for uploading
    s3_client.upload_file(file_name, bucket_name, user+'/'+ file_name)             # Uploadfile to s3
    file_write = open(file_write.name, "a")                   # Reopen close file
    return file_write

In [49]:
# Upload file to s3 every a duration of time
print('Creating file csv')
file_signal = exist_file_from_s3(file_name, user, BUCKET_NAME, s3_client)
update_flag = datetime.now() + timedelta(minutes = 2) 
print("Read data from Serial")
try: 
    while 1:
        # now = datetime.now()
        #if (now.second) == 59:
        #    counter_minute = counter_minute + 1
        data = read_serial.readline()  # Data from serial separate by newline character
        data = data.strip()            # Remove newline character of Arduino IDE 
        file_signal.write(datetime.now().strftime("%H:%M:%S.%f") + ",")    # Record time corresponding to the signal 
        file_signal.write(user + ",")    # Write to column Patient Id
        file_signal.write(data)
        file_signal.write("\n")
        
        #if counter_minute == 10:         # Upload file every
        if datetime.now() > update_flag:
            file_signal = update_file_signal(file_signal, file_name, user, BUCKET_NAME, s3_client)
            print('Uploading File')
            #counter_minute = 0
            update_flag = datetime.now() + timedelta(minutes = 2) 
            
except KeyboardInterrupt:
    file_signal.close()
    print("End")

Creating file csv
downloading file...
opening the file...
Finished!
Read data from Serial
Uploading File
Uploading File
Uploading File
Uploading File
Uploading File
Uploading File
Uploading File
End


In [7]:
# Experiment with datetime
timedelta(hours=1)

datetime.timedelta(0, 3600)

In [9]:
from datetime import datetime, timedelta

dt = datetime.now()
dt

datetime.datetime(2018, 1, 24, 12, 10, 1, 165036)

In [16]:
a = dt + timedelta(hours=1)
a

datetime.datetime(2018, 1, 12, 15, 54, 19, 235280)

In [17]:
b = a.replace(minute=10)
b

datetime.datetime(2018, 1, 12, 15, 10, 19, 235280)

In [25]:
test = datetime.now()
test_1 = test + timedelta(minutes=71)

In [26]:
test

datetime.datetime(2018, 1, 12, 15, 6, 50, 884246)

In [28]:
test_1

datetime.datetime(2018, 1, 12, 16, 17, 50, 884246)

#### Automaticly create new file data if data is collected continuous in many days

In [22]:
current_time = datetime.now()
file_name = current_time.strftime("%H:%M:%S") + '.csv'
file_name

'12:25:51.csv'

In [34]:
# The code will try on minutes as this is faster, the code will be modified on day.
print('Creating file csv')
file_signal = exist_file_from_s3(file_name, user, BUCKET_NAME, s3_client)
update_flag = datetime.now() + timedelta(minutes = 2) 
print("Read data from Serial")
try: 
    while 1:
        data = read_serial.readline()  # Data from serial separate by newline character
        data = data.strip()            # Remove newline character of Arduino IDE 
        file_signal.write(datetime.now().strftime("%H:%M:%S.%f") + ",")    # Record time corresponding to the signal 
        file_signal.write(user + ",")    # Write to column Patient Id
        file_signal.write(data)
        file_signal.write("\n")
        
        if datetime.now() > update_flag:
            file_signal = update_file_signal(file_signal, file_name, user, BUCKET_NAME, s3_client)
            print('Uploading File')
            update_flag = datetime.now() + timedelta(minutes = 2) 
        # New day
        #if (current_time.day != datetime.now().day) or (current_time.month != datetime.now().month):
        if (current_time.minute != datetime.now().minute):
            current_time = datetime.now()
            file_name = current_time.strftime("%H:%M:%S") + '.csv'
            print(file_name)
            file_signal = exist_file_from_s3(file_name, user, BUCKET_NAME, s3_client)
            update_flag = datetime.now() + timedelta(minutes = 2)
        
except KeyboardInterrupt:
    file_signal.close()
    print("End")

Creating file csv
Making file...
Finished!
Read data from Serial
12:30:06.csv
Making file...
Finished!
12:31:00.csv
Making file...
Finished!
12:32:00.csv
Making file...
Finished!
12:33:00.csv
Making file...
Finished!
12:34:00.csv
Making file...
Finished!
12:35:00.csv
Making file...
Finished!
12:36:00.csv
Making file...
Finished!
12:37:00.csv
Making file...
Finished!
End


In [21]:
datetime.now().month

1

In [37]:
if (current_time.day != datetime.now().day) \
or (current_time.month != datetime.now().month):
    print(hello)

## Apply Bluetooth Receiver

In [15]:
import bluetooth
import sys
from bluetooth import *


print "looking for nearby devices..."
nearby_devices = bluetooth.discover_devices(lookup_names = True, flush_cache = True, duration = 20)
print "found %d devices" % len(nearby_devices)

looking for nearby devices...
found 1 devices


In [16]:
print("Find target device")
target_addr = 0
for addr, name in nearby_devices:
    try:
        if name == "Canaria":
            target_addr = addr
    except UnicodeEncodeError:
        name = name.encode('utf-8', 'replace')
        if name == "Canaria":
            target_addr = addr
if target_addr == 0:
    print("Target Device Unavailable")
    sys.exit()

Find target device


In [17]:
port = 1                                            # Declare the port which Bluetooth signal is read
sock = bluetooth.BluetoothSocket(bluetooth.RFCOMM)
sock.connect((target_addr, port))
print("Connected with Bluetooth")

Connected with Bluetooth


In [20]:
update_time = 5           # duration how often the s3 be updated
f = sock.makefile("r+")
print('Creating file csv')
file_signal = exist_file_from_s3(file_name, user, BUCKET_NAME, s3_client)
update_flag = datetime.now() + timedelta(minutes = update_time)
print("Read data from Bluetooth")

data = ""
try:
    while 1:
        #f = sock.makefile("r+")
        data = f.readline()
        #data += sock.recv(10)
        #data_end = data.find('\n') 
        file_signal.write(datetime.now().strftime("%H:%M:%S.%f") + ",")    # Record time corresponding to the signal 
        file_signal.write(user + ",")    # Write to column Patient Id
        print(data)
        file_signal.write(data)
        file_signal.write("\n")
        #data = data[data_end+1:]
        time.sleep(0.01)
        
        # Time to update s3
        if datetime.now() >= update_flag:         # Update the file
            file_signal = update_file_signal(file_signal, file_name, user, BUCKET_NAME, s3_client)
            update_flag = datetime.now() + timedelta(minutes = update_time)
        # New day, program create new data file
        if (current_time.day != datetime.now().day) or (current_time.month != datetime.now().month) \
        or (current_time.year != datetime.now().year):
            file_signal = update_file_signal(file_signal, file_name, user, BUCKET_NAME, s3_client)
            current_time = datetime.now()
            file_name = current_time.strftime("%Y-%m-%d") + '.csv'
            file_signal = exist_file_from_s3(file_name, user, BUCKET_NAME, s3_client)
            update_flag = datetime.now() + timedelta(minutes = update_time)
            
except KeyboardInterrupt:
    file_signal.close()
    sock.close()
    print("End Recording Process")

Creating file csv
Making file...
Finished!
Read data from Bluetooth
-5571

Red:-5567

Red:-5568

Red:-5569

Red:-5570

Red:-5564

Red:-5573

Red:-5571

Red:-5569

Red:-5574

Red:-5568

Red:-5567

Red:-5572

Red:-5570

Red:-5573

Red:-5575

Red:-5572

Red:-5570

Red:-5568

Red:-5570

Red:-5570

Red:-5570

Red:-5569

Red:-5574

Red:-5570

Red:-5568

Red:-5569

Red:-5570

Red:-5568

Red:-5567

Red:-5570

Red:-5569

Red:-5568

Red:-5574

Red:-5573

Red:-5567

Red:-5572

Red:-5572

Red:-5572

Red:-5568

Red:-5571

Red:-5570

Red:-5569

Red:-5570

Red:-5575

Red:-5569

Red:-5571

Red:-5571

Red:-5566

Red:-5569

Red:-5571

Red:-5570

Red:-5572

Red:-5573

Red:-5572

Red:-5569

Red:-5570

Red:-5572

Red:-5575

Red:-5570

Red:-5568

Red:-5571

Red:-5575

Red:-5577

Red:-5572

Red:-5573

Red:-5571

Red:-5571

Red:-5569

Red:-5574

Red:-5571

Red:-5572

Red:-5570

Red:-5567

Red:-5567

Red:-5569

Red:-5573

Red:-5572

Red:-5570

Red:-5569

Red:-5571

Red:-5569

Red:-5574

Red:-5569

Red:-5574

R

Red:-5564

Red:-5567

Red:-5568

Red:-5570

Red:-5568

Red:-5566

Red:-5563

Red:-5562

Red:-5564

Red:-5564

Red:-5565

Red:-5568

Red:-5566

Red:-5567

Red:-5564

Red:-5567

Red:-5564

Red:-5567

Red:-5566

Red:-5566

Red:-5563

Red:-5566

Red:-5569

Red:-5564

Red:-5565

Red:-5564

Red:-5565

Red:-5564

Red:-5569

Red:-5563

Red:-5562

Red:-5559

Red:-5560

Red:-5563

Red:-5568

Red:-5568

Red:-5562

Red:-5564

Red:-5562

Red:-5562

Red:-5567

Red:-5563

Red:-5565

Red:-5565

Red:-5566

Red:-5563

Red:-5563

Red:-5564

Red:-5562

Red:-5562

Red:-5561

Red:-5560

Red:-5565

Red:-5566

Red:-5564

Red:-5564

Red:-5566

Red:-5564

Red:-5563

Red:-5566

Red:-5563

Red:-5565

Red:-5561

Red:-5570

Red:-5560

Red:-5561

Red:-5564

Red:-5561

Red:-5561

Red:-5561

Red:-5563

Red:-5569

Red:-5564

Red:-5559

Red:-5562

Red:-5563

Red:-5562

Red:-5566

Red:-5564

Red:-5560

Red:-5563

Red:-5565

Red:-5561

Red:-5563

Red:-5563

Red:-5558

Red:-5561

Red:-5564

Red:-5566

Red:-5566

Red:-5562


Red:-5567

Red:-5570

Red:-5569

Red:-5575

Red:-5576

Red:-5580

Red:-5576

Red:-5574

Red:-5570

Red:-5571

Red:-5571

Red:-5573

Red:-5573

Red:-5571

Red:-5573

Red:-5569

Red:-5569

Red:-5573

Red:-5570

Red:-5566

Red:-5576

Red:-5568

Red:-5570

Red:-5566

Red:-5569

Red:-5568

Red:-5572

Red:-5575

Red:-5569

Red:-5570

Red:-5574

Red:-5575

Red:-5572

Red:-5573

Red:-5572

Red:-5574

Red:-5569

Red:-5573

Red:-5571

Red:-5566

Red:-5573

Red:-5573

Red:-5567

Red:-5575

Red:-5569

Red:-5570

Red:-5568

Red:-5568

Red:-5571

Red:-5573

Red:-5571

Red:-5572

Red:-5569

Red:-5571

Red:-5575

Red:-5570

Red:-5567

Red:-5570

Red:-5574

Red:-5571

Red:-5568

Red:-5570

Red:-5571

Red:-5570

Red:-5571

Red:-5574

Red:-5568

Red:-5567

Red:-5566

Red:-5570

Red:-5571

Red:-5572

Red:-5572

Red:-5571

Red:-5566

Red:-5573

Red:-5570

Red:-5570

Red:-5572

Red:-5573

Red:-5574

Red:-5572

Red:-5566

Red:-5564

Red:-5569

Red:-5568

Red:-5569

Red:-5572

Red:-5570

Red:-5570

Red:-5569


Red:-5561

Red:-5566

Red:-5558

Red:-5565

Red:-5564

Red:-5564

Red:-5560

Red:-5568

Red:-5561

Red:-5567

Red:-5561

Red:-5567

Red:-5563

Red:-5564

Red:-5560

Red:-5558

Red:-5560

Red:-5560

Red:-5560

Red:-5567

Red:-5564

Red:-5567

Red:-5564

Red:-5562

Red:-5566

Red:-5560

Red:-5564

Red:-5567

Red:-5564

Red:-5562

Red:-5563

Red:-5563

Red:-5566

Red:-5563

Red:-5560

Red:-5564

Red:-5561

Red:-5564

Red:-5561

Red:-5564

Red:-5559

Red:-5559

Red:-5563

Red:-5559

Red:-5561

Red:-5563

Red:-5564

Red:-5566

Red:-5566

Red:-5563

Red:-5561

Red:-5562

Red:-5564

Red:-5561

Red:-5562

Red:-5561

Red:-5564

Red:-5561

Red:-5564

Red:-5567

Red:-5559

Red:-5559

Red:-5565

Red:-5559

Red:-5559

Red:-5561

Red:-5559

Red:-5564

Red:-5559

Red:-5563

Red:-5562

Red:-5559

Red:-5566

Red:-5565

Red:-5562

Red:-5561

Red:-5558

Red:-5559

Red:-5563

Red:-5555

Red:-5560

Red:-5562

Red:-5560

Red:-5560

Red:-5559

Red:-5556

Red:-5559

Red:-5560

Red:-5564

Red:-5560

Red:-5564


Red:-5567

Red:-5563

Red:-5562

Red:-5563

Red:-5567

Red:-5560

Red:-5567

Red:-5566

Red:-5567

Red:-5564

Red:-5563

Red:-5565

Red:-5566

Red:-5562

Red:-5564

Red:-5565

Red:-5559

Red:-5565

Red:-5565

Red:-5566

Red:-5567

Red:-5565

Red:-5564

Red:-5562

Red:-5564

Red:-5560

Red:-5563

Red:-5562

Red:-5563

Red:-5560

Red:-5560

Red:-5566

Red:-5563

Red:-5563

Red:-5563

Red:-5563

Red:-5562

Red:-5564

Red:-5564

Red:-5560

Red:-5560

Red:-5557

Red:-5560

Red:-5566

Red:-5562

Red:-5566

Red:-5563

Red:-5566

Red:-5567

Red:-5556

Red:-5556

Red:-5559

Red:-5558

Red:-5558

Red:-5562

Red:-5566

Red:-5558

Red:-5565

Red:-5561

Red:-5561

Red:-5564

Red:-5562

Red:-5555

Red:-5562

Red:-5563

Red:-5567

Red:-5563

Red:-5557

Red:-5563

Red:-5557

Red:-5559

Red:-5554

Red:-5560

Red:-5558

Red:-5554

Red:-5559

Red:-5558

Red:-5566

Red:-5564

Red:-5561

Red:-5565

Red:-5564

Red:-5560

Red:-5554

Red:-5558

Red:-5560

Red:-5564

Red:-5560

Red:-5563

Red:-5561

Red:-5567


Red:-5557

Red:-5555

Red:-5559

Red:-5559

Red:-5566

Red:-5568

Red:-5564

Red:-5570

Red:-5562

Red:-5565

Red:-5557

Red:-5559

Red:-5558

Red:-5564

Red:-5560

Red:-5566

Red:-5564

Red:-5570

Red:-5568

Red:-5569

Red:-5564

Red:-5555

Red:-5555

Red:-5553

Red:-5559

Red:-5566

Red:-5565

Red:-5564

Red:-5559

Red:-5569

Red:-5565

Red:-5565

Red:-5563

Red:-5559

Red:-5557

Red:-5559

Red:-5556

Red:-5558

Red:-5558

Red:-5558

Red:-5569

Red:-5565

Red:-5569

Red:-5573

Red:-5568

Red:-5565

Red:-5566

Red:-5564

Red:-5559

Red:-5557

Red:-5563

Red:-5560

Red:-5561

Red:-5561

Red:-5567

Red:-5565

Red:-5566

Red:-5569

Red:-5562

Red:-5562

Red:-5559

Red:-5559

Red:-5557

Red:-5558

Red:-5568

Red:-5567

Red:-5567

Red:-5569

Red:-5570

Red:-5562

Red:-5572

Red:-5561

Red:-5564

Red:-5561

Red:-5563

Red:-5560

Red:-5560

Red:-5566

Red:-5563

Red:-5569

Red:-5563

Red:-5561

Red:-5567

Red:-5560

Red:-5559

Red:-5559

Red:-5563

Red:-5560

Red:-5561

Red:-5566

Red:-5562


Red:-5562

Red:-5562

Red:-5566

Red:-5566

Red:-5567

Red:-5566

Red:-5563

Red:-5571

Red:-5566

Red:-5567

Red:-5565

Red:-5566

Red:-5563

Red:-5562

Red:-5566

Red:-5564

Red:-5564

Red:-5563

Red:-5563

Red:-5565

Red:-5570

Red:-5563

Red:-5561

Red:-5565

Red:-5557

Red:-5559

Red:-5565

Red:-5566

Red:-5570

Red:-5564

Red:-5566

Red:-5564

Red:-5566

Red:-5562

Red:-5567

Red:-5561

Red:-5564

Red:-5564

Red:-5562

Red:-5559

Red:-5559

Red:-5563

Red:-5567

Red:-5562

Red:-5563

Red:-5567

Red:-5568

Red:-5564

Red:-5565

Red:-5565

Red:-5564

Red:-5563

Red:-5566

Red:-5567

Red:-5564

Red:-5567

Red:-5565

Red:-5567

Red:-5559

Red:-5561

Red:-5563

Red:-5567

Red:-5569

Red:-5566

Red:-5562

Red:-5564

Red:-5569

Red:-5559

Red:-5571

Red:-5566

Red:-5567

Red:-5566

Red:-5562

Red:-5567

Red:-5568

Red:-5575

Red:-5568

Red:-5560

Red:-5572

Red:-5564

Red:-5569

Red:-5564

Red:-5562

Red:-5563

Red:-5565

Red:-5573

Red:-5564

Red:-5570

Red:-5566

Red:-5568

Red:-5569


In [23]:
from datetime import datetime, timedelta
update_time = 5           # duration how often the s3 be updated
print('Creating file csv')
file_signal = exist_file_from_s3(file_name, user, BUCKET_NAME, s3_client)
update_flag = datetime.now() + timedelta(minutes = update_time)
print("Read data from Bluetooth")
sock.recv(1024)


Creating file csv
Making file...
Finished!
Read data from Bluetooth


In [93]:
f = sock.makefile("r+")

In [ ]:
f.readline()

In [12]:
port = 1
target_addr = '20:17:08:15:12:41'
sock = bluetooth.BluetoothSocket(bluetooth.RFCOMM)

In [13]:
if sock == None:
    print('Error')

f = sock.makefile()
while 1:
    rec = f.readline()
    print(rec)
    print("Yolo")
    time.sleep(0.1)

KeyboardInterrupt: 